In [ ]:
# ONS (Census 2011) statistics for Lewisham.
# Fetches Nomis data releases and prepares them for use in Lewisham ward profiles.
#
# This requires the OA->LSOA lookup table produced by "lbl_boundaries_oa.ipynb"
# as well as the OA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [ ]:
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the Nomis source data.

In [ ]:
# Extract the subset we need.
def filter_nomis_data(d, date_values, geography_values):
  return d[d.date.isin(date_values) & 
           d.geography.isin(geography_values)]

# Reformat Nomis data into a lookup table format:
# clear labelling of the index (geography) column, 
# remove superfluous columns.
def format_nomis_data(d, geo_colname='OA11CD'):
  return d.\
           rename(columns={'geography code': geo_colname}).\
           drop(columns=['date', 'geography'])

# Convenience function to call the above.
def nomis_to_oa(d, years, oa_list):
  return format_nomis_data(filter_nomis_data(d, years, oa_list))

In [ ]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
oa_lookups_dir = f"{project_dir}/lookups/oa"
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
output_dir = f"{project_dir}/health"

In [ ]:
!mkdir -p '{output_dir}'

Downloads
--

In [ ]:
# Health and provision of unpaid care
# Source: 
# https://www.nomisweb.co.uk/census/2011/ks301ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_617_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_health_and_provision_of_unpaid_care_oa11_london.csv"

--2021-11-08 15:19:35--  https://www.nomisweb.co.uk/api/v01/dataset/nm_617_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2212433 (2.1M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/health/census11_health_and_provision_of_unpaid_care_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   2.11M   643KB/s    in 3.4s    

2021-11-08 15:19:51 (643 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/health/census11_health_and_provision_of_unpaid_care_oa11_london.csv’ saved [2212433/2212433]



Lookups
--
Used to match up OAs with their LSOAs, and 2022 electoral wards

In [ ]:
# All OAs and their LSOAs in Lewisham
oa_lsoa_join = pd.read_csv(f"{oa_lookups_dir}/lbl_oa11_lsoa11_msoa11_lad20_rgn20_202012.csv")
oa_lsoa_join = oa_lsoa_join[['OA11CD', 'LSOA11CD']]
oa_lsoa_join.head()

,OA11CD,LSOA11CD
0,E00016277,E01003220
1,E00016278,E01003220
2,E00016285,E01003220
3,E00016257,E01003221
4,E00016263,E01003221


In [ ]:
# All OAs and their 2022 wards in Lewisham
oa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_oa11_wd22_proposed.csv")
oa_wd22_join = oa_wd22_join[['OA11CD', 'WD22CD_proposed']]
oa_wd22_join.head()

,OA11CD,WD22CD_proposed
0,E00016403,E05013721
1,E00016442,E05013721
2,E00016407,E05013721
3,E00016404,E05013721
4,E00016402,E05013721


Process & export
==

In [ ]:
# For filtering of source data
lbl_oa_list = oa_lsoa_join.OA11CD.unique() # All OAs in Lewisham
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['health_and_provision_of_unpaid_care']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_oa11_london.csv")

  # Transform to OA index data
  lbl_oa = nomis_to_oa(d, years, lbl_oa_list)
  lbl_oa.to_csv(f"{output_dir}/lbl_{datname}_oa11.csv", index=False)

  # Aggregate to LSOA level
  lbl_lsoa = aggregate_oa_groups(lbl_oa, oa_lsoa_join, oa_col='OA11CD', group_col='LSOA11CD')
  lbl_lsoa.to_csv(f"{output_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(lbl_lsoa.head())

  # Aggregate to WD22 level
  lbl_wd22 = aggregate_oa_groups(lbl_oa, oa_wd22_join, oa_col='OA11CD', group_col='WD22CD_proposed')
  lbl_wd22.to_csv(f"{output_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(lbl_wd22.head())

'=== health_and_provision_of_unpaid_care ==='

,disability/health/care: All categories: Long-term health problem or disability; measures: Value,disability/health/care: Day-to-day activities limited a lot; measures: Value,disability/health/care: Day-to-day activities limited a little; measures: Value,disability/health/care: Day-to-day activities not limited; measures: Value,disability/health/care: Day-to-day activities limited a lot: Age 16 to 64; measures: Value,disability/health/care: Day-to-day activities limited a little: Age 16 to 64; measures: Value,disability/health/care: Day-to-day activities not limited: Age 16 to 64; measures: Value,disability/health/care: Very good health; measures: Value,disability/health/care: Good health; measures: Value,disability/health/care: Fair health; measures: Value,disability/health/care: Bad health; measures: Value,disability/health/care: Very bad health; measures: Value,disability/health/care: Provides no unpaid care; measures: Value,disability/health/care: Provides 1 to 19 hours unpaid care a week; measures: Value,disability/health/care: Provides 20 to 49 hours unpaid care a week; measures: Value,disability/health/care: Provides 50 or more hours unpaid care a week; measures: Value
LSOA11CD,,,,,,,,,,,,,,,,
E01003189,1670,144,157,1369,77,92,858,741,552,263,99,15,1495,93,28,54
E01003190,1878,136,145,1597,86,95,1043,874,650,229,87,38,1725,72,28,53
E01003191,1494,119,122,1253,80,84,813,680,558,177,65,14,1353,92,25,24
E01003192,1639,173,146,1320,108,86,876,714,541,249,102,33,1497,75,25,42
E01003193,1568,157,141,1270,80,78,868,653,600,197,93,25,1412,84,30,42


,disability/health/care: All categories: Long-term health problem or disability; measures: Value,disability/health/care: Day-to-day activities limited a lot; measures: Value,disability/health/care: Day-to-day activities limited a little; measures: Value,disability/health/care: Day-to-day activities not limited; measures: Value,disability/health/care: Day-to-day activities limited a lot: Age 16 to 64; measures: Value,disability/health/care: Day-to-day activities limited a little: Age 16 to 64; measures: Value,disability/health/care: Day-to-day activities not limited: Age 16 to 64; measures: Value,disability/health/care: Very good health; measures: Value,disability/health/care: Good health; measures: Value,disability/health/care: Fair health; measures: Value,disability/health/care: Bad health; measures: Value,disability/health/care: Very bad health; measures: Value,disability/health/care: Provides no unpaid care; measures: Value,disability/health/care: Provides 1 to 19 hours unpaid care a week; measures: Value,disability/health/care: Provides 20 to 49 hours unpaid care a week; measures: Value,disability/health/care: Provides 50 or more hours unpaid care a week; measures: Value
WD22CD_proposed,,,,,,,,,,,,,,,,
E05013714,10627,957,935,8735,580,571,5675,4821,3572,1478,584,172,9632,539,172,284
E05013715,15544,1038,1035,13471,476,602,9851,8211,5017,1546,578,192,14409,760,164,211
E05013716,16587,859,1003,14725,538,647,11875,8616,5732,1580,501,158,15564,712,139,172
E05013717,16097,1172,1241,13684,544,694,9386,7629,5634,2030,616,188,14523,1000,243,331
E05013718,14937,932,1020,12985,480,626,9555,7690,4959,1604,523,161,13629,851,190,267
